# Confidence Assessment

# DL models

For DL models the confidence for each prediction can be assessed by taking the output of the Softmax function for the predicted class.

In the case of the autoencoder and the siamese network a different approach has been proposed.

# Siamese Network

In [ ]:
import numpy as np

def confidence_siamese(model, template_N, template_S, template_V, test_beat):
    dist_N = model.predict([template_N, test_beat])
    dist_S = model.predict([template_S, test_beat])
    dist_V = model.predict([template_V, test_beat])
    distances = np.concatenate([dist_N, dist_S, dist_V], axis=1)
    y_pred = np.argmin(distances, axis=1)

    return 1-distances[y_pred]

# Autoencoder

In [ ]:
def confidence_autoencoder(original_beat, reconstructed_beat, threshold, max_mae, min_mae):
    mae_beat = np.mean(np.abs(original_beat - reconstructed_beat))
    conf = abs(mae_beat-threshold)
    
    if mae_beat >= threshold:
        conf_norm = conf/abs(max_mae-threshold)
    else:
        conf_norm = conf/abs(min_mae-threshold)

    return conf_norm

# ML models

For ML models confidence for each prediction can be assessed by computing the class probabilities for each model via the ```predict_proba``` method by scikit learn (see report for explanation).

For the SVM a different approach has been proposed.

In [ ]:
def get_confidence_scores(model, X, y):

    # Predict probabilities
    y_pred_prob = model.predict_proba(X)
    # Create a dictionary with predicted probabilities and real labels
    confidence_dict = [{'confidence': y_pred_prob[i].tolist(), 'true_class': y[i]} for i in range(len(X))]

    return confidence_dict

In [ ]:
# Recover the single beats from the test set
# test_beats = []
# for patient in X_test:
#     for beat in patient.single_beats:
#         test_beats.append(beat)
# len(test_beats), len(confidence_scores)

'''Ensure that for the beats in the test set are available to plot the beat and the confidence for each with the following code.'''

In [ ]:
import random
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import numpy as np

def plot_signal_and_confidence(confidence_scores, test_beats, index=None):
    # If no index is provided, select a random one
    if index is None:
        index = random.randint(0, len(test_beats) - 1)

    # Create a figure with two subplots: one for the signal and one for the histogram
    fig, axs = plt.subplots(1, 2, figsize=(10, 5))
    
    # Define class labels
    class_labels = ['N' if label == 0.0 else 'S' if label == 1.0 else 'V' for label in range(len(confidence_scores[index]['confidence']))]

    # Plot the signal
    axs[0].plot(test_beats[index])
    axs[0].set_title(f'Test set beat {index}: Class {class_labels[confidence_scores[index]["true_class"]]}')


    # Generate a color map
    colors = cm.rainbow(np.linspace(0, 1, len(confidence_scores[index]['confidence'])))

    # Plot a horizontal histogram of the confidence
    axs[1].barh(class_labels, confidence_scores[index]['confidence'], color=colors)
    axs[1].set_title('Prediction Confidence')
    axs[1].set_xlabel('Probability')
    axs[1].set_ylabel('Class')

    # Show the plot
    plt.tight_layout()
    plt.show()

# SVM

In [ ]:
def confidence_svm(model, test_beat, max_dist, min_dist):
    dist = model.decision_function(test_beat)
    confidence_score = (dist - min_dist) / (max_dist - min_dist)

    return confidence_score